In [ ]:
!pip install xgboost


In [ ]:
# Bloco 1: Upload e Preparação dos Dados

# Importar as bibliotecas necessárias
import json
from collections import defaultdict
import pandas as pd
from google.colab import files

# Função para carregar os dados
def load_data():
    # Fazer upload do dataset JSON
    uploaded = files.upload()

    # Extrair o nome do arquivo do dataset
    dataset_filename = list(uploaded.keys())[0]

    # Carregar o dataset a partir do arquivo JSON
    with open(dataset_filename, 'r') as f:
        dataset = json.load(f)

    # Converter o dataset para um DataFrame para manipulação mais fácil
    df = pd.DataFrame(dataset)
    return df

# Função para estruturar os cantos por tempo e parte
def structure_data(df):
    cantos_por_tempo = defaultdict(lambda: defaultdict(list))
    for canto in df.to_dict(orient='records'):
        cantos_por_tempo[canto['tempo']][canto['parte']].append(canto)

    # Separar um canto de cada categoria para validação
    validacao = []
    for tempo, partes in cantos_por_tempo.items():
        for parte, cantos in partes.items():
            if cantos:  # Verifica se há cantos disponíveis
                validacao.append(cantos.pop(0))  # Remove o primeiro canto para validação

    # Organizar o restante dos cantos como dados de treino
    treino = [canto for partes in cantos_por_tempo.values() for cantos in partes.values() for canto in cantos]

    return pd.DataFrame(treino), pd.DataFrame(validacao)

# Execução das funções para carregar e preparar os dados
df = load_data()
df_train, df_val = structure_data(df)


Saving dataset.json to dataset (2).json


In [ ]:
# Bloco 2: Treinamento do Modelo

# Importar bibliotecas necessárias para vetorização e modelo
from sklearn.feature_extraction.text import TfidfVectorizer
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV

# Função para treinar o modelo
def train_model(df_train, df_val):
    # Vetorização dos textos das letras dos cantos usando TF-IDF
    vectorizer = TfidfVectorizer(max_features=5000)
    X_train = vectorizer.fit_transform(df_train['lyric'])
    X_val = vectorizer.transform(df_val['lyric'])

    # Definindo as saídas (labels) como as categorias de tempo litúrgico
    y_train = df_train['tempo']
    y_val = df_val['tempo']

    # Codificar as labels para inteiros
    label_encoder = LabelEncoder()
    y_train_encoded = label_encoder.fit_transform(y_train)
    y_val_encoded = label_encoder.transform(y_val)

    # Definição do modelo base
    model = XGBClassifier(eval_metric='mlogloss')

    # Definição da grade de parâmetros para ajuste
    param_grid = {
        'n_estimators': [50, 100, 200],
        'max_depth': [3, 5, 7, 10],
        'learning_rate': [0.01, 0.1, 0.2],
        'subsample': [0.5, 0.7, 1.0],
        'colsample_bytree': [0.5, 0.7, 1.0],
        'min_child_weight': [1, 3, 5]
    }

    # Configurar a busca em grade
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=3, verbose=1)

    # Treinamento da busca em grade
    grid_search.fit(X_train, y_train_encoded)

    # Resultados do melhor modelo encontrado
    print("Melhores parâmetros:", grid_search.best_params_)
    print("Melhor acurácia:", grid_search.best_score_)

    # Avaliação do melhor modelo no conjunto de validação
    y_pred_encoded = grid_search.predict(X_val)

    # Decodificar as previsões para os rótulos originais
    y_pred = label_encoder.inverse_transform(y_pred_encoded)

    # Exibir as métricas de desempenho do modelo
    print("Acurácia no conjunto de validação:", accuracy_score(y_val, y_pred))
    print("\nRelatório de Classificação:\n", classification_report(y_val, y_pred))

# Chamar a função para treinar o modelo
train_model(df_train, df_val)

Fitting 3 folds for each of 972 candidates, totalling 2916 fits
Melhores parâmetros: {'colsample_bytree': 0.7, 'learning_rate': 0.01, 'max_depth': 3, 'min_child_weight': 1, 'n_estimators': 50, 'subsample': 1.0}
Melhor acurácia: 0.43922127255460586
Acurácia no conjunto de validação: 0.45

Relatório de Classificação:
               precision    recall  f1-score   support

     advento       0.40      0.50      0.44         4
       comum       0.25      0.25      0.25         4
       natal       1.00      0.50      0.67         4
      pascoa       0.75      0.75      0.75         4
    quaresma       0.20      0.25      0.22         4

    accuracy                           0.45        20
   macro avg       0.52      0.45      0.47        20
weighted avg       0.52      0.45      0.47        20

